In [1]:
import pandas as pd
import re # Регулярные выражения.
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import pymorphy2 # Морфологический анализатор.
import datetime # Новости будем перебирать по дате.
from collections import Counter # Не считать же частоты самим.
import math # Корень квадратный.
import numpy

In [2]:
train = pd.read_csv('data/train.csv', sep='\t', index_col='id')

In [3]:
test = pd.read_csv('data/test.csv', sep='\t', index_col='id')

In [4]:
other = pd.read_csv('data/other.csv', sep='\t')

In [5]:
train.head()

,name,description,target
id,,,
0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0
3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0
4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1


In [6]:
train.describe()

,target
count,200000.000000
mean,0.467820
std,0.498965
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [7]:
test.head()

,name,description
id,,
200000,Дизайнер-консультант мебели,<p><strong>Обязанности:</strong></p> <ul> <li>...
200001,Продавец-консультант (ТЦ на Пушкина),<p><strong>Обязанности</strong>:</p> <p>∙ конс...
200002,Менеджер по продажам,<p>Торговый Дом «Форт» это ведущая компания Пе...
200003,Продавец-консультант в магазин одежды (ТЦ Волн...,<p><strong>Требуются продавцы консультанты в м...
200004,Специалист по охране труда,<strong>Обязанности:</strong> <ul> <li> <p>осу...


In [8]:
other.head()

,name,description
0,Специалист научно-производственного отдела,"<p>Образование - Среднее специальное, высшее</..."
1,Оператор по отгрузке товара в 1С (ТЗ),<p><strong>Обязанности:</strong></p> <ul> <li>...
2,Менеджер по персоналу,<strong>Обязанности:</strong> <ul> <li>Подбор ...
3,Ведущий бухгалтер по учёту заработной платы,<strong>Обязанности:</strong> <ul> <li> <p>Нач...
4,Инженер-расчетчик в строительный отдел,<p><strong>Требования:</strong></p> <ul> <li>у...


In [9]:
test.describe()

,name,description
count,170179,170179
unique,61614,124537
top,Менеджер по продажам,"<div>Для каждого, кто хочет работать и зарабат..."
freq,5363,17084


In [10]:
# test.groupby('name').describe()

In [11]:
y = train.target
train = train.drop('target', axis=1)

In [12]:
def preproc_data(df_input):
    df_output = df_input.copy()
    
    df_output = df_output.drop('description', axis=1)
    df_output['name'] = df_output['name'].str.lower()
    df_output['name'] = df_output['name'].str.replace(u'[^а-яa-z -]','')
    
    return df_output

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

train_data = preproc_data(train)
test_data = preproc_data(test)

In [14]:
cv = CountVectorizer()
cv.fit_transform(train_data.append(test_data)['name'])

pickle.dump(cv, open('cv.pckl', 'wb'))

In [15]:

cv.vocabulary_

{'заведующий': 12609,
 'отделомсекцией': 21831,
 'магазин': 17597,
 'york': 5863,
 'уручье': 31348,
 'наладчик': 20050,
 'станков': 28854,
 'манипуляторов': 17855,
 'пу': 25184,
 'разработчик': 25549,
 'криптограф': 16433,
 'фрезеровщик': 32037,
 'мерчендайзерпродавец': 18763,
 'консультант': 15660,
 'мастер': 18090,
 'по': 23123,
 'эксплуатации': 33494,
 'зданий': 13063,
 'торговый': 30547,
 'представитель': 23993,
 'менеджер': 18557,
 'продажам': 24504,
 'работе': 25312,
 'ключевыми': 15043,
 'клиентами': 14872,
 'posm': 4102,
 'backend': 442,
 'developer': 1405,
 'nodejsnosql': 3648,
 'продавец': 24449,
 'кассир': 14405,
 'орск': 21667,
 'ул': 31135,
 'станиславского': 28844,
 'специалист': 28519,
 'технической': 30211,
 'поддержки': 23230,
 'истра': 13961,
 'гвардейской': 10125,
 'дивизии': 11575,
 'программист': 24381,
 'директор': 11693,
 'магазина': 17598,
 'кемерово': 14621,
 'свободы': 27239,
 'pr': 4123,
 'оборудования': 20976,
 'администратор': 6357,
 'младший': 19214,
 'ана

In [16]:
cv_train_data = cv.transform(train_data.name)
cv_test_data = cv.transform(test_data.name)

X_train, X_test, y_train, y_test = train_test_split(cv_train_data, y, test_size=0.33, random_state=42)
lr = LogisticRegression()

lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

print(roc_auc_score(y_test, lr.predict_proba(X_test)[:,1]))
print(log_loss(y_test, lr.predict_proba(X_test)[:,1]))

0.9832912738074123
0.1543136003066057


In [18]:
predict = lr.predict_proba(cv_test_data)

result_predict = pd.DataFrame(data={'id': test.index, 'target': predict[:,1]})
result_predict.to_csv('dsaponenko_06_predict.csv', index=False)